# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

É um problema de classificação, pois temos duas classes, 'precisa de ajuda' (quando parece certo que o aluno não se formará) e 'não precisa de ajuda', e uma delas deve ser atribuída para cada estudante.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
# subtraindo 1 pois a última coluna é o target
n_features = len(student_data.columns) - 1 

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = (float(n_passed) / n_students) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
# Usando estratificação para garantir mesma proporção de classes na base de treino e na de teste
X_train, X_test, y_train, y_test = \
    train_test_split(X_all, y_all, stratify=y_all, test_size=num_test, train_size=num_train, random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?

- What makes this model a good candidate for the problem, given what you know about the data?

In [6]:
pd.set_option('display.max_columns', 48)
X_all.describe()

,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,Medu,Fedu,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,0.883544,0.116456,0.526582,0.473418,16.696203,0.222785,0.777215,0.711392,0.288608,0.103797,0.896203,2.749367,2.521519,0.149367,0.086076,0.356962,0.260759,0.146835,0.050633,0.045570,0.549367,0.281013,0.073418,0.367089,0.275949,0.091139,0.265823,0.227848,0.691139,0.081013,1.448101,2.035443,0.334177,0.129114,0.612658,0.458228,0.508861,0.794937,0.949367,0.832911,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861
std,0.321177,0.321177,0.499926,0.499926,1.276043,0.416643,0.416643,0.453690,0.453690,0.305384,0.305384,1.094735,1.088201,0.356902,0.280832,0.479711,0.439606,0.354391,0.219525,0.208814,0.498188,0.450064,0.261152,0.482622,0.447558,0.288172,0.442331,0.419976,0.462610,0.273201,0.697505,0.839240,0.743651,0.335751,0.487761,0.498884,0.500555,0.404260,0.219525,0.373528,0.472300,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096
min,0.000000,0.000000,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,16.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,17.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000
75%,1.000000,0.000000,1.000000,1.000000,18.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,4.000000,3.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,4.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0

**Answer:**

Observando a tabela anterior, é possível notar que o intervalo de possíveis valores assumidos dentro de cada atributo não é fixo. Por exemplo, *age* $\in [ 15, 22 ]$ enquanto *absences* $\in [0, 75]$. Alguns algoritmos, como o KNN utilizando distância euclidiana, são sensíveis a isso; assim, na escolha dos modelos, é relevante considerar a necessidade de efetuar algum tipo de pré-processamento nos dados utilizando, por exemplo, `sklearn.preprocessing.StandardScaler` ou `sklearn.preprocessing.Normalizer`.

## Análise dos modelos

    
### Gradient Boosting

* aplicação: melhoria da extrapolação de dados de uma amostragem para toda a população, confome visto em http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.298.987&rep=rep1&type=pdf; com isso, é possível reduzir custos ao não fazer um censo completo.
* prós: funciona bem em casos não-lineares http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3885826/ e não exige conhecimento a priori sobre a distribuição estatística dos dados.
* contras: http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3885826/ para um bom treinamento, é necessário um learning_rate adequado (usualmente < 1), o que pode fazer com que o número de iterações aumente consideravelmente. Isso gera um maior consumo de memória, utilizada para armazenar todos os parâmetros dos weak learners que serão gerados. Além disso, na etapa de classificação, cada entrada deve ser processada por cada um dos modelos internos treinados anteriormente, aumentando o tempo de resposta.
* por que é um bom modelo para o problema?: dada a quantidade de atributos (30), me parece que há um número suficiente de dimensões para a sequência de weak leaners conseguirem tratar dos casos difíceis.

### SVM

* aplicação: pode ser usado como filtro antispam: http://trec.nist.gov/pubs/trec15/papers/hit.spam.final.final.pdf.
* prós: por tratar de otimização em um conjunto convexo (http://www.cs.cmu.edu/~epxing/Class/10701-08s/recitation/svm.pdf), SVM sempre encontra a solução ótima dados o conjunto de dados de treinamento e a parametrização, evitando mínimos locais. Além disso, como utiliza o conceito de maximização da margem entre classes juntamente com o fator de regularização $C$, há um maior controle sobre overfitting. Permite dar pesos diferentes para cada classe, de modo a aumentar a taxa de cobertura. Por último, utilizando o truque de kernel, consegue separar espaços não-lineares.
* contras: primeiramente, a otimização dos hiper-parâmetros é trabalhosa, pois depende da escolha do kernel, seus respectivos parâmetros e o fator de regularização. Em segundo lugar, trata-se de um algoritmo que não escala bem com um volume maior de dados.
* por que é um bom modelo para o problema?: basicamente, a quantidade de amostras e o número de atributos é pequeno, tornando o uso do algoritmo viável, desde que seja feita a normalização dos dados.

Em vários tutoriais, é recomendado que os atributos tenham média 0 e variância unitária; usarei o módulo `StandardScaler` para isso.

### KNN

* aplicação: auxílio em diagnósticos médicos baseado no histórico do paciente: https://www.researchgate.net/publication/270163293_Application_of_k-Nearest_Neighbour_Classification_in_Medical_Data_Mining.
* prós: trata de casos não-lineares; novos dados do conjunto de treino podem ser acrescentados sem que um processamento de treino tenha que ser realizado desde o início.
* contras: muito suscetível à maldição da dimensionalidade; bases muito grandes tornam o armazenamento mais difícil (dependente de mais memória ou de uma boa estrutura para guardar e recuperar os dados em disco, ou de uma arquitetura distribuída) e o processamento da classificação ou regressão mais demorado.
* por que é um bom modelo para o problema?: o número de atributos após a conversão para representação *one-hot* (48) não parece ser grande o suficiente para gerar problemas com a maldição da dimensionalidade (isso é altamente subjetivo).

Como a métrica de distância padrão é a euclidiana, é interessante normalizar os dados, do contrário atributos com valores absolutos e intervalos maiores irão dominar o aprendizado das classes. Usarei o módulo `StandardScaler`.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

## Importante

Como o objetivo do modelo é encontrar estudantes com maiores chances de não conseguirem se formar, substitui o `pos_label` no cálculo do F1 score para a classe '`no`'. Utilizar '`yes`' trouxe alguns problemas na otimização via `GridSearch`, conforme mostrado no final do relatório. Além disso, para ajudar nessa explicação, acrescentei também a geração da matriz de confusão para cada chamada ao método `predict_labels`.

In [22]:
from sklearn.metrics import confusion_matrix
import sys

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def print_confusion_matrix(y, y_pred):
    print 'Confusion matrix:'
    labels = np.unique(y)
    cm = confusion_matrix(y, y_pred, labels)
    
    s = ''
    for i in range(0, len(labels)):
        for j in range(0, len(labels)):
            s = s + ' | %3s as %3s %4d'%(labels[i], labels[j], cm[i][j])
        s = s + ' |\n'
    
    sys.stdout.write(s)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    print_confusion_matrix(target.values, y_pred)

    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='no')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    name = clf.get_params()['clf'].__class__.__name__
    print "Training a {} using a training set size of {}. . .".format(name, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [18]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# usado para normalização
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# TODO: Initialize the three models
clf_A_std = Pipeline([('clf', GradientBoostingClassifier(random_state=0))])
clf_B_std = Pipeline([('scl', StandardScaler()), ('clf', SVC(random_state=0))])
clf_C_std = Pipeline([('scl', StandardScaler()), ('clf', KNeighborsClassifier())])


# TODO: Set up the training set sizes
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)

X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train
y_train_300 = y_train
    
for clf in [clf_A_std, clf_B_std, clf_C_std]:    
    train_predict(clf, X_train_100, y_train_100, X_test, y_test)
    train_predict(clf, X_train_200, y_train_200, X_test, y_test)    
    train_predict(clf, X_train_300, y_train_300, X_test, y_test)
    print ""    

Training a GradientBoostingClassifier using a training set size of 100. . .
Trained model in 0.0448 seconds
Confusion matrix:
 |  no as  no   34 |  no as yes    0 |
 | yes as  no    0 | yes as yes   66 |
Made predictions in 0.0004 seconds.
F1 score for training set: 1.0000.
Confusion matrix:
 |  no as  no   15 |  no as yes   16 |
 | yes as  no   18 | yes as yes   46 |
Made predictions in 0.0003 seconds.
F1 score for test set: 0.4687.
Training a GradientBoostingClassifier using a training set size of 200. . .
Trained model in 0.0570 seconds
Confusion matrix:
 |  no as  no   65 |  no as yes    0 |
 | yes as  no    0 | yes as yes  135 |
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Confusion matrix:
 |  no as  no   12 |  no as yes   19 |
 | yes as  no   11 | yes as yes   53 |
Made predictions in 0.0003 seconds.
F1 score for test set: 0.4444.
Training a GradientBoostingClassifier using a training set size of 300. . .
Trained model in 0.0676 seconds
Confusion matrix

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gradient Boosting **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0448           |     0.0003             |       1.0000     |     0.4867      |
| 200               |        0.0570           |     0.0006             |       1.0000     |     0.4444      |
| 300               |        0.0676           |     0.0003             |       0.9355     |     0.3600      |

** Classifer 2 - SVM **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0014           |     0.0008             |       0.8852     |     0.2105      |
| 200               |        0.0035           |     0.0012             |       0.7129     |     0.2564      |
| 300               |        0.0070           |     0.0018             |       0.7389     |     0.2500      |

** Classifer 3 - KNN **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0006           |     0.0010             |       0.6667     |     0.4286      |
| 200               |        0.0007           |     0.0017             |       0.5310     |     0.3137      |
| 300               |        0.0010           |     0.0025             |       0.5257     |     0.3750      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Para a escolha do melhor modelo, podemos comparar as características de cada algoritmo, juntamente com os 3 dados mais importantes das tabelas geradas anteriormente:

* tempo de treinamento
* tempo para gerar previsão
* F1 score (na base de tamanho 300)

***KNN***

É o algoritmo de menor tempo de treinamento. No entanto, esse valor não é diretamente comparável pois trata-se basicamente de uma cópia dos dados de treinamento para um cache em memória; a lógica de inferência da classe está totalmente contida na busca dos vizinhos mais próximos do vetor de entrada e na maneira que os pesos de cada vizinho entram no cálculo do resultado final.

Possui o melhor F1, mas pior tempo de classificação.

***SVM***

Tem o segundo tempo de treinamento, mas com um sério agravante: aparenta ter complexidade exponencial, pois o tempo aproximadamente dobra para cada incremento de 100 linhas no conjunto de entrada.

É o segundo também no tempo de predição e o pior em relação ao F1 score.

***Gradient Boosting***

É o pior em termos de tempo de treinamento, mas a função que determina sua complexidade em cpu é, aparentemente, linear. Possui ainda o menor tempo de predição e segundo melhor F1 score.

***Conclusão***

O modelo escolhido é o Gradient Boosting, pois embora tenha o F1 score um pouco abaixo do KNN, possui o menor tempo para gerar predições. Ou seja, uma vez treinado, é o mais rápido tanto para processamento batch (se fosse feito uma análise para todos os estudantes de um dado país, por exemplo) quanto online (um site em que o estudante digitaria seus dados e seria informado se precisaria ou não de reforço).

Em um cenário em que a ordem de grandeza da base de treinamento fosse aumentada, SVM não seria viável por culpa de sua complexidade exponencial e KNN poderia ter problemas por limitações do tamanho do seu cache. No entanto, sendo (aparentemente) linear, o treinamento no modelo Gradient Boosting ainda seria viável.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Nota mental: honestamente, não acho possível reduzir o modelo de Gradient Boosting em uma explicação sem matemática. Procurei exemplos em vários lugares diferentes e, invariavelmente, caí em 1 de 3 cenários possíveis: 1. uma abstração tão simplificada que poderia ser aplicada a basicamente qualquer coisa; 2. uma explicação simples e coerente, exceto pelo problema de que o modelo era o AdaBoost, não GB; 3. um artigo baseado no capítulo de *boosting* do *The Elements of Statistical Learning* e tão indecifrável quanto.

Primeiramente, precisamos entender o que é uma árvore de decisão. Ela é um conjunto de perguntas em sequência feitas sobre determinados atributos de uma dada entidade. Em seu treinamento, são efetuadas repetidas iterações de, dado um certo conjunto de entidades, encontrar qual pergunta que deve ser feita de maneira a tentar dividi-lo em 2 subconjuntos de tamanho o mais próximo possível, até o momento em que uma condição de parada é atingida (o tamanho do conjunto é menor do que um limite ou todos os elementos do conjunto pertencem a uma única classe). Durante a etapa de classificação, basta observar a pergunta no topo e, baseado na resposta, ir para a próxima pergunta. Esse processo é repetido até chegar em um ponto em que, ao invés de haver uma pergunta, encontraremos um nó que indica a classe resposta.

A base do Gradient Boosting é gerar *N* classificadores em sequência (por padrão, são árvores de decisão). Para cada instante *t*, o resultado do modelo *t-1* é utilizado da seguinte maneira: compara-se o valor predito *y_pred* com o valor esperado *y* via uma função de erro. Essa função deve ser derivável para que possamos encontrar o seu gradiente. A árvore no instante *t* irá, então, usar como alvo esse gradiente e não mais o valor *y* para cada uma das entradas utilizadas no treino para *t*. Ao final, teremos uma árvore que representa a função que minimiza a função de erro original.

O uso do gradiente da função de erro garante implicitamente que entradas difíceis de serem classificadas terão um fator de ajuste (baseado no gradiente) maior do que as entradas fáceis. Ao iterarmos várias vezes, aumentamos a chance de obtermos uma árvore que aprenda a classificar tanto os elementos fáceis quanto os difíceis.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

# Importante

Ao trocar o `pos_label` para '`no`', houve casos em que a chamada para `f1_score` gerava um warning devido a uma divisão
por zero. Isso acontecia pois certos modelos geravam apenas a classe '`yes`', zerando tanto a precisão quanto cobertura para a classe '`no`'. Criei o método `f1_score_no_warning`, que devolve 0 caso nenhuma das predições seja igual ao `pos_label` passado como parâmetro.

In [24]:
def f1_score_no_warning(y_true, y_pred, **kwargs):
    if (kwargs['pos_label'] is None):
        kwargs['pos_label'] = 1
        
    if (kwargs['pos_label'] in np.unique(y_pred)):
        return f1_score(y_true, y_pred, **kwargs)
    else:
        return 0

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'n_estimators' : [50, 100, 200, 300],
              'max_depth' : [1, 2, 3, 5, 8, 13, 21, 34],
              'learning_rate' : [0.001, 0.1, 0.2, 0.5],
              'subsample' : [1.0, 0.8, 0.7, 0.6, 0.5]
             }

# TODO: Initialize the classifier
clf = GradientBoostingClassifier(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score_no_warning, pos_label='no')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf,
                        param_grid=parameters,
                        scoring=f1_scorer,
                        n_jobs=8)

# TODO: Fit the grid search object to the training data and find the optimal parameters
start = time()
grid_obj = grid_obj.fit(X_train, y_train)
end = time()
print "Grid search in in {:.4f} seconds.".format(end - start)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Grid search in in 77.4805 seconds.
Confusion matrix:
 |  no as  no   99 |  no as yes    0 |
 | yes as  no    0 | yes as yes  201 |
Made predictions in 0.0014 seconds.
Tuned model has a training F1 score of 1.0000.
Confusion matrix:
 |  no as  no   11 |  no as yes   20 |
 | yes as  no   13 | yes as yes   51 |
Made predictions in 0.0005 seconds.
Tuned model has a testing F1 score of 0.4000.


A execução do GridSearch sobre um SVC buscando aumentar o F1 score baseado na classe '`yes`' obteve um classificador que não devolveu '`no`' nos testes. Essa é a razão pela qual fiz as principais alterações no enunciado original:

* Utilizar `pos_label`='`no`'.
* Enfatizar o problema de usar pos_label='yes' ao mostrar a matriz de confusão juntamente com o score f1.

Entendo que, se tivesse mantido `pos_label`='`yes`', o F1 score obtido seria melhor do que o encontrado no código acima, mas potencialmente enviesado para gerar classificações `yes`, causado pela distribuição das classes não ser balanceada. Os códigos abaixo evidenciam o problema:

In [26]:
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    print_confusion_matrix(target.values, y_pred)

    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

param_range = [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]

# TODO: Create the parameters list you wish to tune
param_grid = [{'clf__C' : param_range,
               'clf__kernel' : ['linear']},
              {'clf__C' : param_range,
               'clf__gamma': param_range,
               'clf__kernel': ['rbf']}               
             ]

# TODO: Initialize the classifier
pipe = Pipeline([
        ('scl', StandardScaler()),
        ('clf', SVC(random_state=0)) 
        ])

# clf = GradientBoostingClassifier(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=pipe,
                        param_grid=param_grid,
                        scoring=f1_scorer,
                        n_jobs=8,
                        cv=3)

# TODO: Fit the grid search object to the training data and find the optimal parameters
start = time()
grid_obj = grid_obj.fit(X_train, y_train)
end = time()
print "Grid search in in {:.4f} seconds.".format(end - start)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Grid search in in 38.3662 seconds.
Confusion matrix:
 |  no as  no    0 |  no as yes   99 |
 | yes as  no    0 | yes as yes  201 |
Made predictions in 0.0033 seconds.
Tuned model has a training F1 score of 0.8024.
Confusion matrix:
 |  no as  no    0 |  no as yes   31 |
 | yes as  no    0 | yes as yes   64 |
Made predictions in 0.0013 seconds.
Tuned model has a testing F1 score of 0.8050.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

Antes do `GridSearch`
F1 de treino: 0.9355
F1 de teste: 0.3600

Após `GridSearch`
F1 de treino: 1.0000
F1 de teste: 0.4000

Houve uma melhora do F1 de teste e de treino. No entanto, a grande diferença entre os F1 scores de treino e teste, tanto antes quanto depois do `GridSearch` sugere que o classificador sofre de *overfitting*.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.